In [0]:
from huggingface_hub import notebook_login
# Login to Huggingface to get access to the model
notebook_login()

In [0]:
%pip install xformers

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 11.3 MB/s eta 0:00:00
     ━━

In [0]:
dbutils.library.restartPython()

In [0]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# it is suggested to pin the revision commit hash and not change it for reproducibility because the uploader might change the model afterwards; you can find the commmit history of llamav2-7b-chat in https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/commits/main
model = "meta-llama/Llama-2-7b-chat-hf"
revision = "08751db2aca9bf2f7f80d2e516117a53d7450235"

tokenizer = AutoTokenizer.from_pretrained(model, padding_side="left")
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    revision=revision,
    return_full_text=False
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [0]:
pipeline.tokenizer.pad_token_id = tokenizer.eos_token_id

# COMMAND ----------

# Define prompt template to get the expected features and performance for the chat versions. See our reference code in github for details: https://github.com/facebookresearch/llama/blob/main/llama/generation.py#L212

DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
PROMPT_FOR_GENERATION_FORMAT = """
<s>[INST]<<SYS>>
{system_prompt}
<</SYS>>

{instruction}
[/INST]
""".format(
    system_prompt=DEFAULT_SYSTEM_PROMPT,
    instruction="{instruction}"
)


In [0]:
def gen_text(prompts, use_template=False, **kwargs):
    if use_template:
        full_prompts = [
            PROMPT_FOR_GENERATION_FORMAT.format(instruction=prompt)
            for prompt in prompts
        ]
    else:
        full_prompts = prompts

    if "batch_size" not in kwargs:
        kwargs["batch_size"] = 1
    
    # the default max length is pretty small (20), which would cut the generated output in the middle, so it's necessary to increase the threshold to the complete response
    if "max_new_tokens" not in kwargs:
        kwargs["max_new_tokens"] = 1000

    # configure other text generation arguments, see common configurable args here: https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.GenerationConfig
    kwargs.update(
        {
            "pad_token_id": tokenizer.eos_token_id,  # Hugging Face sets pad_token_id to eos_token_id by default; setting here to not see redundant message
            "eos_token_id": tokenizer.eos_token_id,
        }
    )

    outputs = pipeline(full_prompts, **kwargs)
    outputs = [out[0]["generated_text"] for out in outputs]

    return outputs

In [0]:
results = gen_text(["what is the recipe for masala dosa?"])
print(results[0])



Masala dosa is a popular South Indian dish that is made with a fermented rice and lentil batter, filled with spiced potatoes and onions, and served with sambar and chutney. Here is a recipe for masala dosa:

Ingredients:

For the dosa batter:

* 1 cup of rice
* 1 cup of lentils (split red lentils or toor dal)
* 1/2 cup of water
* 1/4 teaspoon of salt
* 1/4 teaspoon of sugar
* 1/2 teaspoon of active dry yeast
* 1/4 teaspoon of asafetida (optional)

For the filling:

* 2 medium-sized potatoes, peeled and grated
* 1 medium-sized onion, finely chopped
* 1/4 teaspoon of ground cumin
* 1/4 teaspoon of ground coriander
* 1/4 teaspoon of turmeric powder
* Salt, to taste
* 2 tablespoons of oil

Instructions:

1. To make the dosa batter, soak the rice and lentils in water for 4-5 hours or overnight. Drain the water and grind the rice and lentils into a smooth batter using a mixer or blender. Add the salt, sugar, yeast, and asafetida (if using) to the batter and mix well. Cover the batter with 

In [0]:
results = gen_text(["""Translate the given text to Hindi langauge
                 '''A large language model is a type of artificial intelligence (AI) model that is trained on a large dataset of text to generate language outputs that are coherent and natural-sounding'''
"""])
print(results[0])

In [0]:
results = gen_text(["""change the given text to Hindi langauge
                 '''A large language model is a type of artificial intelligence (AI) model that is trained on a large dataset of text to generate language outputs that are coherent and natural-sounding'''
"""], temperature=0.1, max_new_tokens=200, use_template=True)
print(results[0])

एक बड़ी संस्कृति में संचालित हैं एक संस्कृति का प्रारंभिक संचालन हैं, जो एक बड़ा संकलन का संचालन करता हैं, जो एक नाम का संचालन करता हैं, जो संचालन के साथ संचालित हैं, जो संचालन के साथ संच



Translation in Hindi:
'''एक बड़ा भाषा मॉडल एक प्रकार का इंटेलिजेंस मॉडल है | इसे बड़ा संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन का संकलन 

In [0]:
long_input = """Provide a concise summary of the below passage.

Hannah Arendt was one of the seminal political thinkers of the twentieth century. The power and originality of her thinking was evident in works such as The Origins of Totalitarianism, The Human Condition, On Revolution and The Life of the Mind. In these works and in numerous essays she grappled with the most crucial political events of her time, trying to grasp their meaning and historical import, and showing how they affected our categories of moral and political judgment. What was required, in her view, was a new framework that could enable us to come to terms with the twin horrors of the twentieth century, Nazism and Stalinism. She provided such framework in her book on totalitarianism, and went on to develop a new set of philosophical categories that could illuminate the human condition and provide a fresh perspective on the nature of political life.

Although some of her works now belong to the classics of the Western tradition of political thought, she has always remained difficult to classify. Her political philosophy cannot be characterized in terms of the traditional categories of conservatism, liberalism, and socialism. Nor can her thinking be assimilated to the recent revival of communitarian political thought, to be found, for example, in the writings of A. MacIntyre, M. Sandel, C. Taylor and M. Walzer. Her name has been invoked by a number of critics of the liberal tradition, on the grounds that she presented a vision of politics that stood in opposition some key liberal principles. There are many strands of Arendt’s thought that could justify such a claim, in particular, her critique of representative democracy, her stress on civic engagement and political deliberation, her separation of morality from politics, and her praise of the revolutionary tradition. However, it would be a mistake to view Arendt as an anti-liberal thinker. Arendt was in fact a stern defender of constitutionalism and the rule of law, an advocate of fundamental human rights (among which she included not only the right to life, liberty, and freedom of expression, but also the right to action and to opinion), and a critic of all forms of political community based on traditional ties and customs, as well as those based on religious, ethnic, or racial identity.

Arendt’s political thought cannot, in this sense, be identified either with the liberal tradition or with the claims advanced by a number of its critics. Arendt did not conceive of politics as a means for the satisfaction of individual preferences, nor as a way to integrate individuals around a shared conception of the good. Her conception of politics is based instead on the idea of active citizenship, that is, on the value and importance of civic engagement and collective deliberation about all matters affecting the political community. If there is a tradition of thought with which Arendt can be identified, it is the classical tradition of civic republicanism originating in Aristotle and embodied in the writings of Machiavelli, Montesquieu, Jefferson, and Tocqueville. According to this tradition politics finds its authentic expression whenever citizens gather together in a public space to deliberate and decide about matters of collective concern. Political activity is valued not because it may lead to agreement or to a shared conception of the good, but because it enables each citizen to exercise his or her powers of agency, to develop the capacities for judgment and to attain by concerted action some measure of political efficacy."""

def get_num_tokens(text):
    inputs = tokenizer(text, return_tensors="pt").input_ids.to("cuda")
    return inputs.shape[1]

print('number of tokens for input:', get_num_tokens(long_input))

results = gen_text([long_input], max_new_tokens=150, use_template=True)
print(results[0])

number of tokens for input: 777
Hannah Arendt was a prominent political thinker of the 20th century, known for her works such as "The Origins of Totalitarianism" and "The Human Condition." Her political philosophy is distinct from traditional conservatism, liberalism, and socialism, and cannot be classified into the recent revival of communitarian thought. While some critics have invoked her name as a critique of liberal tradition, Arendt was actually a defender of constitutionalism and the rule of law, advocating for fundamental human rights and civic engagement. Her conception of politics is based on active citizenship and collective deliberation, rather than individual preferences or integration around a shared conception of the


In [0]:
long_input = """Translate the given text to Hindi langauge
                 A large language model is a type of artificial intelligence (AI) model that is trained on a large dataset of text to generate language outputs that are coherent and natural-sounding
"""

def get_num_tokens(text):
    inputs = tokenizer(text, return_tensors="pt").input_ids.to("cuda")
    return inputs.shape[1]

print('number of tokens for input:', get_num_tokens(long_input))

results = gen_text([long_input], max_new_tokens=150, use_template=True)
print(results[0])

number of tokens for input: 52
एक बड़ी भाषा मॉडल एक प्रकार के संज्ञानुसार काउंट्रीबुक एक बड़ा संकलन का प्रारंभ करता है, जहां एक बड़ा संकलन का प्रारंभ करता है, जहां 
